In [1]:
import requests
import json
import pandas as pd
from config2 import api_key
from config2 import gkey
import csv
import gmaps

In [2]:
gmaps.configure(api_key=gkey)

In [3]:
beer_csv = pd.read_csv("addresses2.csv", delimiter=",")
beer_csv_df=pd.DataFrame(beer_csv)
beer_csv_df.head()

,Name,Address,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,City
0,10 Barrel Brewing Co.,1501 E St,San Diego,CA 92101,NaN,NaN,San Diego
1,2Kids Brewing Co.,8680 Miralani Drive #123,San Diego,CA 92126,NaN,NaN,San Diego
2,13 Point Brewing Company,8035 Broadway,Lemon Grove,CA 91945,NaN,NaN,Lemon Grove
3,32 North Brewing Co.,8655 Production Avenue Suite A,San Diego,CA 92121,NaN,NaN,San Diego
4,Abnormal Beer Company,16990 Via Tazon Suite 124,San Diego,CA 92127,NaN,NaN,San Diego


In [4]:
biz_name = beer_csv_df["Name"]
addr = beer_csv_df["Address"]
city = beer_csv_df["City"]
addy = addr.values.tolist()
biz = biz_name.values.tolist()
cities = city.values.tolist()
len(cities)

152

In [5]:
# for index, row in beer_csv_df.iterrows():
#     biz, addy = row['Name'], row['Address']
#     print(a)

In [6]:
ids = []
name = []
address = []
lat = []
lng = []
rating = []
review = []
x = -1
addy 
biz

skippedb = []
skippeda = []
for b, a, c in zip(biz, addy, cities):
    x += 1
    print(f"printing {b} || {x} of {len(biz)} ")
# for b, a, in beer_csv_df.iterrows():
    headers = {'Authorization': 'Bearer %s' % api_key}
    params = {'name':b,'address1':a,'city':c, 'state':'Ca', 'country':'US'}
    url = "https://api.yelp.com/v3/businesses/matches"
    try:
        response = requests.get(url, headers=headers, params=params)
        response_json = response.json()
        response_json
        idz = response_json["businesses"][0]["id"]
        addresses = response_json["businesses"][0]["location"]["address1"]
        lats = response_json["businesses"][0]["coordinates"]["latitude"]
        lngs = response_json["businesses"][0]["coordinates"]["longitude"]
        names = response_json["businesses"][0]["name"]
        name.append(names)
        address.append(addresses)
        lat.append(lats)
        lng.append(lngs)
        ids.append(idz)
    except:
        skippedb.append(b)
        skippeda.append(a)
        print('Skipping:', b)
        continue

for b, a, c, i, in zip(biz, addy, cities, ids):
    params = {'name':b,'address1':a,'city':c, 'state':'Ca', 'country':'US'}
    url = "https://api.yelp.com/v3/businesses/"
    response = requests.get(url + i, headers=headers, params=params)
    response_json = response.json()
    ratings = response_json["rating"]
    reviews = response_json["review_count"]
    review.append(reviews)
    rating.append(ratings)
        

printing 10 Barrel Brewing Co. || 0 of 152 
printing 2Kids Brewing Co. || 1 of 152 
printing 13 Point Brewing Company || 2 of 152 
printing 32 North Brewing Co. || 3 of 152 
printing Abnormal Beer Company || 4 of 152 
printing AleSmith Brewing Company || 5 of 152 
printing Align Brewing Co. || 6 of 152 
printing Alpine Beer Company || 7 of 152 
printing Amplified Ale Works || 8 of 152 
printing Amplified Ale Works || 9 of 152 
printing Arcana Brewing Co. || 10 of 152 
printing Attitude Brewing || 11 of 152 
printing Automatic Brewing Company || 12 of 152 
printing Aztec Brewing Company || 13 of 152 
printing Back Street Brewery || 14 of 152 
printing Bagby Beer Comapny || 15 of 152 
printing Ballast Point Brewing & Spirits - Home Brew Mart || 16 of 152 
printing Ballast Point Brewing & Spirits - Little Italy || 17 of 152 
printing Ballast Point Brewing & Spirits - Miramar || 18 of 152 
printing Ballast Point Brewing & Spirits - Scripps Ranch || 19 of 152 
printing Barn Brewery || 20 of

In [7]:
skippedb

["Hodad's Brewing Co.",
 'Home Brewing Co',
 'Jamul Brewing Co.',
 'Kensington Brewing Company',
 'Mason Ale Works',
 'Mother Earth Brew Company',
 'Resident Brewing Co.']

In [8]:
yelp_brew_dict = {"Name":name, "Review Count":review, "Rating":rating}
yelp_brew_df = pd.DataFrame(yelp_brew_dict)
yelp_brew_df["Lat"] = lat
yelp_brew_df["lng"] = lng
yelp_brew_df.head(2)

,Name,Review Count,Rating,Lat,lng
0,10 Barrel Brewing - San Diego,364,4.5,32.714559,-117.150229
1,2kids Brewing Co,86,4.0,32.896564,-117.137428


In [59]:
lat_list = yelp_brew_df["Lat"].tolist()
lng_list = yelp_brew_df["lng"].tolist()
rat_list = yelp_brew_df["Rating"].astype(float)
coordinates = list(zip(lat_list, lng_list))
locations = yelp_brew_df[["Lat", "lng"]].astype(float)
brew_name = yelp_brew_df["Name"].tolist()

In [80]:
locations
figure_layout = {'width': '900px',
                 'height': '500px',
                 'border': '1px solid black',
                 'padding': '1px',
                 'margin': '0 auto 0 auto',
                 'Bold':True
                }
fig = gmaps.figure(layout=figure_layout)
markers = gmaps.marker_layer(coordinates, 
                             hover_text=[f"{brew} \n Rating:{rating}" for brew, rating in zip(brew_name, rat_list)], 
                             label='', 
                             info_box_content=[f"Brewery Rating: {rating}" for rating in rat_list], 
                             display_info_box=None
                            )
symbol_layer.markers[0].scale = 5
fig.add_layer(markers)
#_____________________________

fig
#__________________________________

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

In [64]:
# brewery_layer = gmaps.symbol_layer(
#     locations, fill_color='red',
#     stroke_color='red', scale=2,
#     info_box_content=[f"Brewery Name: {brewery}" for brewery in brew_name]
# )


# fig = gmaps.figure()
# fig.add_layer(brewery_layer)
# fig

In [9]:
a = -1
rating = []
review = []
test = []
for x, c in zip(biz, cities):
    a += 1
    params = {'term':x, 'city':c, "location":"CA"}
    url = "https://api.yelp.com/v3/businesses/search"
    response = requests.get(url, headers=headers, params=params)
    response_json = response.json()
#     cit = response_json["businesses"][0]["location"]["city"]
#     test.append(cit)

#     review.append(reviews)
#     rating.append(ratings)
# review

    response_json

KeyboardInterrupt: 